### references : 
https://computingforgeeks.com/install-postgresql-14-on-ubuntu-jammy-jellyfish/

### run postgres : 
sudo -u postgres psql

- CREATE ROLE admin WITH LOGIN SUPERUSER CREATEDB CREATEROLE PASSWORD 'Passw0rd';

### check user : 
\du

- create database wa1;
- create user wu1 with encrypted password 'Passw0rd';
- grant all privileges on database wa1 to wu1;

In [1]:
import pandas as pd
import psycopg2

hostStr = '172.20.151.20'
dbPort = 5432
dbStr = 'wa1'
uNameStr = 'wu1'
dbPassStr = 'Passw0rd'

CONNECT_DB = "host={0} port={1} dbname={2} user={3} password={4}".format(hostStr, dbPort, dbStr, uNameStr, dbPassStr)

try:
    conn = psycopg2.connect(CONNECT_DB)

    # get a cursor object from the connection
    cur = conn.cursor()

    # use the cursor object to run SQL commands to perform operations like fetch rows, insert rows, update rows, delete rows etc.
except (Exception, psycopg2.Error) as error:
    print("Error while interacting with PostgreSQL...\n", error)
    records = 0
finally:
    if(conn):
        # close the cursor object to avoid memory leaks
        cur.close()
        # close the connection object also
        conn.close()

In [2]:
create_table_query = '''CREATE TABLE IF NOT EXISTS transactions (
        step INT,
        customer VARCHAR(20) PRIMARY KEY,
        age VARCHAR(10),
        gender VARCHAR(10),
        zipcodeOri VARCHAR(20),
        merchant VARCHAR(20),
        zipMerchant VARCHAR(10),
        category VARCHAR(50),
        amount FLOAT,
        fraud INT
    ); '''

try:
    # Make connection to db
    cxn = psycopg2.connect(CONNECT_DB)

    # Create a cursor to db
    cur = cxn.cursor()
    
    # Send sql query to request
    cur.execute(create_table_query)
    records = cxn.commit()

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
    
finally:
    #closing database connection.
    if(cxn):
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")

print(f'Records:\n {records}')

PostgreSQL connection is closed
Records:
 None


In [3]:
## Masukkan data transactions
try:
    cxn = psycopg2.connect(CONNECT_DB)
    cur = cxn.cursor()

    with open('./Dataset/transactions.csv', 'r') as f:
        next(f)  # skip header
        for line in f:
            data = line.strip().split(',')
            source = data[0].strip("'")  # remove quotes

            # Insert with conflict handling
            cur.execute("""INSERT INTO transactions (step, customer, age, gender, zipcodeOri, merchant, zipMerchant, category, amount, fraud) 
                   VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                   ON CONFLICT (customer) DO NOTHING""", data)

    cxn.commit()
    print("transactions table populated (skipping duplicates)")

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL", error)

finally:
    if cxn:
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")
        print("transactions table populated")

transactions table populated (skipping duplicates)
PostgreSQL connection is closed
transactions table populated


In [ ]:
create_table_query = '''CREATE TABLE IF NOT EXISTS source (
    Source VARCHAR(20) PRIMARY KEY,
    Target VARCHAR(20),
    Weight FLOAT,
    typeTrans VARCHAR(50),
    fraud INT
); '''

try:
    # Make connection to db
    cxn = psycopg2.connect(CONNECT_DB)

    # Create a cursor to db
    cur = cxn.cursor()
    
    # Send sql query to request
    cur.execute(create_table_query)
    records = cxn.commit()

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
    
finally:
    #closing database connection.
    if(cxn):
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")

print(f'Records:\n {records}')

PostgreSQL connection is closed
Records:
 None


In [ ]:
## Masukkan data Source
try:
    cxn = psycopg2.connect(CONNECT_DB)
    cur = cxn.cursor()

    with open('./Dataset/source.csv', 'r') as f:
        next(f)  # skip header
        for line in f:
            data = line.strip().split(',')
            source = data[0].strip("'")  # remove quotes

            # Insert with conflict handling
            cur.execute("""INSERT INTO source (source, target, weight, typetrans, fraud) 
                        VALUES (%s, %s, %s, %s, %s) 
                        ON CONFLICT (source) DO NOTHING""", data)

    cxn.commit()
    print("source table populated (skipping duplicates)")

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL", error)

finally:
    if cxn:
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")
        print("source table populated")

source table populated (skipping duplicates)
PostgreSQL connection is closed
source table populated


### Cleaning Data

In [6]:
def pandas_db_server_fetch(sql_query):
    try:
        # Make connection to db
        cxn = psycopg2.connect(CONNECT_DB)

        # Send sql query to request and create dataframe
        df = pd.read_sql(sql_query, cxn)

    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)

    finally:
        #closing database connection.
        if(cxn):
            cxn.close()
            print("PostgreSQL connection is closed")
        return df

In [24]:
transactions_df =  pandas_db_server_fetch('''SELECT * FROM transactions''')
transactions_df.head()

PostgreSQL connection is closed


C:\Users\Joseph\AppData\Local\Temp\ipykernel_21928\2075102856.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, cxn)


,step,customer,age,gender,zipcodeori,merchant,zipmerchant,category,amount,fraud
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0


In [26]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8224 entries, 0 to 8223
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   step         8224 non-null   int64  
 1   customer     8224 non-null   object 
 2   age          8224 non-null   object 
 3   gender       8224 non-null   object 
 4   zipcodeori   8224 non-null   object 
 5   merchant     8224 non-null   object 
 6   zipmerchant  8224 non-null   object 
 7   category     8224 non-null   object 
 8   amount       8224 non-null   float64
 9   fraud        8224 non-null   int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 642.6+ KB


In [27]:
transactions_df.isnull().sum()

step           0
customer       0
age            0
gender         0
zipcodeori     0
merchant       0
zipmerchant    0
category       0
amount         0
fraud          0
dtype: int64

In [25]:
# Menghilangkan tanda kutip pada isi kolom
cols_to_clean = ['customer', 'age', 'gender', 'zipcodeori', 'merchant', 'zipmerchant', 'category']

for col in cols_to_clean:
    transactions_df[col] = transactions_df[col].str.replace("'", "")

# Menghilangkan duplikat
transactions_df.drop_duplicates(inplace=True)

# Menampilkan 5 baris pertama
transactions_df.head()

,step,customer,age,gender,zipcodeori,merchant,zipmerchant,category,amount,fraud
0,0,C1093826151,4,M,28007,M348934600,28007,es_transportation,4.55,0
1,0,C352968107,2,M,28007,M348934600,28007,es_transportation,39.68,0
2,0,C2054744914,4,F,28007,M1823072687,28007,es_transportation,26.89,0
3,0,C1760612790,3,M,28007,M348934600,28007,es_transportation,17.25,0
4,0,C757503768,5,M,28007,M348934600,28007,es_transportation,35.72,0


In [29]:
transactions_df['age'] = pd.to_numeric(transactions_df['age'], errors='coerce')
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8224 entries, 0 to 8223
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   step         8224 non-null   int64  
 1   customer     8224 non-null   object 
 2   age          8206 non-null   float64
 3   gender       8224 non-null   object 
 4   zipcodeori   8224 non-null   object 
 5   merchant     8224 non-null   object 
 6   zipmerchant  8224 non-null   object 
 7   category     8224 non-null   object 
 8   amount       8224 non-null   float64
 9   fraud        8224 non-null   int64  
dtypes: float64(2), int64(2), object(6)
memory usage: 642.6+ KB


In [36]:
transactions_df['age'].isna().sum()

18

In [37]:
transactions_df['age'].fillna(value=0, inplace=True)

In [38]:
transactions_df['age'] = transactions_df['age'].astype('int')

In [39]:
transactions_df.head()

,step,customer,age,gender,zipcodeori,merchant,zipmerchant,category,amount,fraud
0,0,C1093826151,4,M,28007,M348934600,28007,es_transportation,4.55,0
1,0,C352968107,2,M,28007,M348934600,28007,es_transportation,39.68,0
2,0,C2054744914,4,F,28007,M1823072687,28007,es_transportation,26.89,0
3,0,C1760612790,3,M,28007,M348934600,28007,es_transportation,17.25,0
4,0,C757503768,5,M,28007,M348934600,28007,es_transportation,35.72,0


In [42]:
customer_counts = transactions_df['customer'].value_counts()

# Temukan jumlah data duplikat
jumlah_duplikat = customer_counts[customer_counts > 1].sum()

# Cetak jumlah data duplikat
print(f"Jumlah data duplikat pada kolom 'customer': {jumlah_duplikat}")

Jumlah data duplikat pada kolom 'customer': 8224


In [43]:
transactions_df = transactions_df.drop_duplicates(subset='customer')

In [44]:
customer_counts = transactions_df['customer'].value_counts()

# Temukan jumlah data duplikat
jumlah_duplikat = customer_counts[customer_counts > 1].sum()

# Cetak jumlah data duplikat
print(f"Jumlah data duplikat pada kolom 'customer': {jumlah_duplikat}")

Jumlah data duplikat pada kolom 'customer': 0


In [45]:
transactions_df.head()

,step,customer,age,gender,zipcodeori,merchant,zipmerchant,category,amount,fraud
0,0,C1093826151,4,M,28007,M348934600,28007,es_transportation,4.55,0
1,0,C352968107,2,M,28007,M348934600,28007,es_transportation,39.68,0
2,0,C2054744914,4,F,28007,M1823072687,28007,es_transportation,26.89,0
3,0,C1760612790,3,M,28007,M348934600,28007,es_transportation,17.25,0
4,0,C757503768,5,M,28007,M348934600,28007,es_transportation,35.72,0


In [46]:
transactions_df.to_csv("transaction_clean.csv", index=False, header=False)

In [41]:
source_df =  pandas_db_server_fetch('''SELECT * FROM source''')
source_df.head()

PostgreSQL connection is closed


C:\Users\Joseph\AppData\Local\Temp\ipykernel_21928\2075102856.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, cxn)


,source,target,weight,typetrans,fraud
0,'C1093826151','M348934600',4.55,'es_transportation',0
1,'C352968107','M348934600',39.68,'es_transportation',0
2,'C2054744914','M1823072687',26.89,'es_transportation',0
3,'C1760612790','M348934600',17.25,'es_transportation',0
4,'C757503768','M348934600',35.72,'es_transportation',0


In [15]:
# Menghilangkan tanda kutip pada isi kolom
cols_to_clean = ['source', 'target', 'typetrans']

for col in cols_to_clean:
    source_df[col] = source_df[col].str.replace("'", "")

# Menghilangkan duplikat
source_df.drop_duplicates(inplace=True)

# Menampilkan 5 baris pertama
source_df.head()

,source,target,weight,typetrans,fraud
0,C1093826151,M348934600,4.55,es_transportation,0
1,C352968107,M348934600,39.68,es_transportation,0
2,C2054744914,M1823072687,26.89,es_transportation,0
3,C1760612790,M348934600,17.25,es_transportation,0
4,C757503768,M348934600,35.72,es_transportation,0


In [16]:
source_df.to_csv("source_clean.csv", index=False, header=False)